## make_pediction
* predict weekly sales 

In [ ]:
import psycopg2 as pg
import pandas as pd
import pandas.io.sql as pd_sql

from sqlalchemy import create_engine
import psycopg2 
import io

import pickle

import pandas as pd
import numpy as np

import matplotlib.pylab as plt
%matplotlib inline
import datetime as dt
import pickle
from sklearn.preprocessing import StandardScaler

connect to database

In [6]:
connection_args = {
    'host': '34.220.165.163', # current aws instance public IP
    'user': 'ubuntu',    
    'dbname': 'ubuntu',   
    'port': 5432         
}

connection = pg.connect(**connection_args)
cur = connection.cursor()

make prediciton

In [29]:
def predict_sales(deptID, storeID, date,isHoliday,temperature,fuel_price,markdown1,markdown2,markdown3,markdown4,markdown5,CPI,unemployment):
    
    # retrieve model from database
    coeffs = pd.read_sql_query('select * from model_coeffs where dept='+str(deptID)+' and store='+str(storeID),con=connection)
    col_means = pd.read_sql_query('select * from col_means where dept=' +str(deptID)+' and store='+str(storeID),con=connection)
    col_stds=pd.read_sql_query('select * from col_stds where dept='+str(deptID)+' and store='+str(storeID),con=connection)
    sarima_pred=pd.read_sql_query('select * from sarima_prediction where store='+str(storeID)+' and dept='+str(deptID)+' and year='+str(date.year)+' and month='+str(date.month)+' and day='+str(date.day),con=connection)['sarima_predicted_scaled_sales']
    
    # scale input
    input_vars = [temperature,fuel_price,markdown1,markdown2,markdown3,markdown4,markdown5,CPI,unemployment]
    scaled_input_vars = [(input_vars[i]-col_means.loc[0][i])/col_stds.loc[0][i] for i in range(len(input_vars))]
                          
                          
    # get prediction from model                      
    X=[1]+[isHoliday]+scaled_input_vars+[not isHoliday]+[sarima_pred]
    scaled_prediction = sum([X[i]*coeffs.loc[0][i] for i in range(len(X))])
    prediction = scaled_prediction*col_stds['weekly_sales']+col_means['weekly_sales']
    return prediction.iloc[0]
    
    

In [30]:
test=predict_sales(1,1,dt.datetime(2012, 11,19),True,68, 2.8, 1000,1000,1500,4200, 7000, 215,5.5)

In [31]:
test

21627.519716795905